# 📦 Instalação de Dependências

In [ ]:
!pip install duckdb -q

---

## 🔢 Bloco 1: Tipos Numéricos

DuckDB suporta diversos tipos numéricos, desde pequenos inteiros até decimais de alta precisão.

In [ ]:
import duckdb
con = duckdb.connect(database=':memory:')

# Inteiros assinados
con.execute("""
CREATE TABLE numeros (
    pequeno TINYINT,     -- -128 a 127
    medio INTEGER,       -- -2B a 2B
    grande BIGINT,       -- muito grande
    enorme HUGEINT       -- extremamente grande
);

INSERT INTO numeros VALUES (127, 1000000, 9223372036854775807, 123456789012345678901234567890);
SELECT * FROM numeros;
""").fetchall()

# Inteiros não-assinados
con.execute("""
CREATE TABLE contadores (
    id UINTEGER,
    visitas UBIGINT
);

INSERT INTO contadores VALUES (1, 18446744073709551615);
SELECT * FROM contadores;
""").fetchall()

# Ponto flutuante
con.execute("""
CREATE TABLE medicoes (
    temperatura FLOAT,   -- 32-bit
    precisao DOUBLE      -- 64-bit
);

INSERT INTO medicoes VALUES (36.5, 3.14159265358979323846);
SELECT * FROM medicoes;
""").fetchall()

# Decimal (precisão exata)
con.execute("""
CREATE TABLE financeiro (
    preco DECIMAL(10, 2),  -- 10 dígitos, 2 decimais
    taxa DECIMAL(5, 4)     -- 5 dígitos, 4 decimais
);

INSERT INTO financeiro VALUES (12345.67, 0.0525);
SELECT * FROM financeiro;
""").fetchall()

print("✅ Tipos numéricos demonstrados")

---

## 📝 Bloco 2: Tipos Texto e Boolean

Trabalhando com strings e valores lógicos.

In [ ]:
import duckdb
con = duckdb.connect(':memory:')

# Tipos texto (VARCHAR, TEXT, STRING são equivalentes)
con.execute("""
CREATE TABLE textos (
    nome VARCHAR,
    descricao TEXT,
    comentario STRING
);

INSERT INTO textos VALUES ('Alice', 'Desenvolvedora', 'Especialista em Python');
SELECT * FROM textos;
""").fetchall()

# VARCHAR com limite
con.execute("""
CREATE TABLE usuarios (
    username VARCHAR(50),
    email VARCHAR(255)
);
""").fetchall()

# Boolean
con.execute("""
CREATE TABLE configuracoes (
    ativo BOOLEAN,
    verificado BOOL,
    publico LOGICAL
);

INSERT INTO configuracoes VALUES (true, false, 1);
SELECT * FROM configuracoes;
""").fetchall()

print("✅ Tipos texto e boolean demonstrados")

---

## 📅 Bloco 3: Tipos Temporais

Data, hora, timestamps e intervalos.

In [ ]:
import duckdb
con = duckdb.connect(':memory:')

# DATE
con.execute("""
CREATE TABLE eventos (
    nome VARCHAR,
    data DATE
);

INSERT INTO eventos VALUES ('Lançamento', '2024-01-15');
SELECT * FROM eventos;
""").fetchall()

# TIME
con.execute("""
CREATE TABLE horarios (
    abertura TIME,
    fechamento TIME
);

INSERT INTO horarios VALUES ('09:00:00', '18:00:00');
SELECT * FROM horarios;
""").fetchall()

# TIMESTAMP
con.execute("""
CREATE TABLE logs (
    mensagem VARCHAR,
    timestamp TIMESTAMP
);

INSERT INTO logs VALUES ('Início', '2024-01-15 10:30:00');
SELECT * FROM logs;
""").fetchall()

# TIMESTAMP WITH TIME ZONE
con.execute("""
CREATE TABLE eventos_globais (
    evento VARCHAR,
    quando TIMESTAMP WITH TIME ZONE
);

INSERT INTO eventos_globais VALUES ('Webinar', '2024-01-15 10:00:00-03:00');
SELECT * FROM eventos_globais;
""").fetchall()

# INTERVAL
result = con.execute("""
SELECT
    '2024-01-15'::DATE + INTERVAL '7 days' as uma_semana,
    '2024-01-15 10:00:00'::TIMESTAMP + INTERVAL '2 hours' as mais_tarde;
""").fetchall()

print("✅ Tipos temporais demonstrados")

---

## 🔐 Bloco 4: Tipos Especiais (BLOB, UUID, JSON)

Dados binários, identificadores únicos e estruturas JSON.

In [ ]:
import duckdb
con = duckdb.connect(':memory:')

# BLOB (dados binários)
con.execute("""
CREATE TABLE arquivos (
    nome VARCHAR,
    conteudo BLOB
);

INSERT INTO arquivos VALUES ('imagem.png', '\xAB\xCD\xEF'::BLOB);
SELECT * FROM arquivos;
""").fetchall()

# UUID
con.execute("""
CREATE TABLE usuarios (
    id UUID,
    nome VARCHAR
);

INSERT INTO usuarios VALUES (uuid(), 'Alice'), (uuid(), 'Bob');
SELECT * FROM usuarios;
""").fetchall()

# JSON
con.execute("""
CREATE TABLE documentos (
    id INTEGER,
    dados JSON
);

INSERT INTO documentos VALUES
(1, '{"nome": "Alice", "idade": 30}'),
(2, '{"produto": "Laptop", "preco": 1200.50}');

SELECT
    id,
    dados->>'$.nome' as nome,
    dados->>'$.idade' as idade
FROM documentos;
""").fetchall()

print("✅ Tipos especiais demonstrados")

---

## 📋 Bloco 5: Tipos Compostos (ARRAY, LIST, STRUCT, MAP)

Estruturas de dados complexas.

In [ ]:
import duckdb
con = duckdb.connect(':memory:')

# ARRAY (tamanho fixo)
con.execute("""
CREATE TABLE vetores (
    coordenadas INTEGER[3]
);

INSERT INTO vetores VALUES ([1, 2, 3]);
SELECT * FROM vetores;
""").fetchall()

# LIST (tamanho variável)
con.execute("""
CREATE TABLE listas (
    numeros INTEGER[]
);

INSERT INTO listas VALUES ([1, 2, 3]), ([10, 20, 30, 40]);

SELECT
    numeros[1] as primeiro,
    array_length(numeros) as tamanho,
    list_sum(numeros) as soma
FROM listas;
""").fetchall()

# STRUCT (objeto com campos nomeados)
con.execute("""
CREATE TABLE pessoas (
    id INTEGER,
    endereco STRUCT(rua VARCHAR, cidade VARCHAR, cep VARCHAR)
);

INSERT INTO pessoas VALUES
(1, {'rua': 'Rua A', 'cidade': 'São Paulo', 'cep': '01000-000'});

SELECT id, endereco.rua, endereco.cidade FROM pessoas;
""").fetchall()

# MAP (chave → valor)
con.execute("""
CREATE TABLE configs (
    id INTEGER,
    opcoes MAP(VARCHAR, INTEGER)
);

INSERT INTO configs VALUES (1, map(['theme', 'font_size'], [1, 14]));

SELECT id, opcoes['theme'] as tema FROM configs;
""").fetchall()

print("✅ Tipos compostos demonstrados")

---

## 🔄 Bloco 6: Conversão de Tipos (CAST)

Convertendo entre diferentes tipos de dados.

In [ ]:
import duckdb
con = duckdb.connect(':memory:')

# CAST explícito
result = con.execute("""
SELECT
    CAST('42' AS INTEGER) as int1,
    CAST(3.14 AS INTEGER) as int2,
    '42'::INTEGER as int3,
    '2024-01-15'::DATE as data,
    'true'::BOOLEAN as bool;
""").fetchall()

print("CAST:", result)

# TRY_CAST (retorna NULL em erro)
result = con.execute("""
SELECT
    TRY_CAST('abc' AS INTEGER) as erro,
    TRY_CAST('42' AS INTEGER) as sucesso;
""").fetchall()

print("TRY_CAST:", result)

# Verificar tipo
result = con.execute("""
SELECT
    typeof(42) as tipo_int,
    typeof('texto') as tipo_varchar,
    typeof([1, 2, 3]) as tipo_lista;
""").fetchall()

print("typeof:", result)
print("✅ Conversões demonstradas")

# Capitulo 07 Tipos Dados Duckdb

Notebook gerado automaticamente a partir do código fonte python.
